# Data Validation and Cleaning

## Load Data

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import Counter

# ========= CONFIG =========
# กำหนด root path ของโฟลเดอร์โปรเจกต์
ROOT_DIR = Path("/Users/thanwaratkeratipasuwat/Desktop/dsi314")

# พาธข้อมูลต้นทาง (Raw Data)
RAW_DIR  = ROOT_DIR / "Raw data"

BKF_PATH  = RAW_DIR / "bkf_eflora_species_dedup.csv"   # BKF หลัง dedup
#OUT7_PATH = RAW_DIR / "output (7).csv"                 # จาก WFO map tools
#CAND_PATH = RAW_DIR / "candidates (3).csv"             # candidates เพิ่มเติม

# พาธผลลัพธ์ (Processed Data)
OUT_DIR   = ROOT_DIR / "Processed data"
#OUT_DIR.mkdir(parents=True, exist_ok=True)  # สร้างโฟลเดอร์ถ้ายังไม่มี

# กำหนดชื่อไฟล์ผลลัพธ์
# OUT_CLEAN_PATH   = OUT_DIR / "bkf_full_after_scrape.csv"        # ข้อมูลที่เติมชื่อแล้ว
# OUT_VALID_PATH   = OUT_DIR / "bkf_full_after_clean.csv"         # หลัง validation + cleaning
# OUT_REVIEW_PATH  = OUT_DIR / "bkf_multicand_review_queue.csv"   # แถวที่ต้องตรวจ manual

# แสดงพาธทั้งหมดเพื่อตรวจสอบ
print("📁 Paths Configuration:")
print(f"BKF_PATH         : {BKF_PATH}")
#print(f"OUT7_PATH        : {OUT7_PATH}")
#print(f"CAND_PATH        : {CAND_PATH}")
print(f"OUT_DIR          : {OUT_DIR}")
# print(f"OUT_CLEAN_PATH   : {OUT_CLEAN_PATH}")
# print(f"OUT_VALID_PATH   : {OUT_VALID_PATH}")
# print(f"OUT_REVIEW_PATH  : {OUT_REVIEW_PATH}")

📁 Paths Configuration:
BKF_PATH         : /Users/thanwaratkeratipasuwat/Desktop/dsi314/Raw data/bkf_eflora_species_dedup.csv
OUT_DIR          : /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data


In [2]:
# ========= HELPERS =========
def norm(x):
    if pd.isna(x): return ""
    return str(x).replace("\u00A0", " ").strip().lower()

def find_col(cols, candidates):
    """หาคอลัมน์แบบทนทานต่อเคสตัวพิมพ์"""
    cols_list = list(cols)
    # หาแบบตรงชื่อก่อน
    for c in candidates:
        if c in cols_list:
            return c
    # หาแบบ lower-case map
    lower_map = {c.lower(): c for c in cols_list}
    for c in candidates:
        if c.lower() in lower_map:
            return lower_map[c.lower()]
    return None

def nonempty_series_like(df, col):
    """คืน Series ของคอลัมน์ (ถ้าไม่มีให้คืนซีรีส์ว่าง) และแปลง nan/None เป็น '' """
    if col is None or col not in df.columns:
        return pd.Series("", index=df.index)
    return (df[col].astype(str)
                 .str.strip()
                 .replace({"nan": "", "None": "", "NaT": ""}))

def wfo_clean(x):
    """ทำความสะอาด WFO -> รูปแบบ wfo-xxxx (ตัวพิมพ์เล็ก, ตัดอักขระแปลก)"""
    if pd.isna(x): return ""
    s = str(x).strip()
    if s == "" or s.lower() in {"none","nan"}:
        return ""
    s2 = s.lower()
    if not s2.startswith("wfo-"):
        s2 = "wfo-" + s2
    s2 = re.sub(r"[^a-z0-9\-]", "", s2)
    return s2

def prefer(left, right):
    """เลือก left ถ้าไม่ว่าง มิฉะนั้นใช้ right"""
    ls = left.astype(str).str.strip()
    rs = right.astype(str).str.strip()
    return np.where(ls != "", ls, rs)

def resolve_eff(df, colname, suffix):
    """คืนชื่อคอลัมน์ที่มีอยู่จริงหลัง merge (เช็คว่ามีการเติม suffix หรือไม่)"""
    if not colname:
        return None
    if f"{colname}{suffix}" in df.columns:
        return f"{colname}{suffix}"
    return colname if colname in df.columns else None

In [3]:
# ========= LOAD =========
df_bkf = pd.read_csv(BKF_PATH)
# df_o7  = pd.read_csv(OUT7_PATH)
# df_c   = pd.read_csv(CAND_PATH)

display(df_bkf.head(3))#, df_o7.head(3), df_c.head(3))


,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id
0,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum siamense (Craib) Tardieu,This is currently accepted.,"PENINSULAR: Nakhon Si Thammarat, Songkhla (type).","Known from 3 localities: the type-locality, an...","In small mats on damp sandy ground, edge of ma...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:33.566587+00:00,Myriophyllum siamense (Craib) Tardieu,NaN
1,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum tetrandrum Roxb.,This is currently accepted.,SOUTH-WESTERN: Prachuap Khiri Khan (Bang Sapha...,"E India (type), Indochina, Malay Peninsula.","In rather shallow, open water of ditches, cana...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:36.633398+00:00,Myriophyllum tetrandrum Roxb.,wfo-0001261381
2,2,Haloragaceae,2 Haloragis,Haloragis,2.0,Haloragis micrantha (Thunb.) R.Br. ex Sieb. & ...,Gonocarpus micranthus,NORTH-EASTERN: Loei (Phu Kradueng).,"India, S & E China, N Vietnam, Hainan, Formosa...","In marshy mountain turf, moist places along mo...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:50.049475+00:00,Gonocarpus micranthus,wfo-0000715064


In [4]:
# ========= IDENTIFY KEY COLUMNS =========
bkf_spec = find_col(df_bkf.columns, ["specific_name"])
bkf_wfo  = find_col(df_bkf.columns, ["wfo_id"])
bkf_url  = find_col(df_bkf.columns, ["species_url"])
bkf_full = find_col(df_bkf.columns, ["wfo_full_name","wfo_name","full_name","accepted_name"])

if bkf_spec is None:
    raise KeyError("ไม่พบคอลัมน์ specific_name ใน BKF")

if bkf_wfo is None:
    # ถ้า BKF ไม่มีคอลัมน์ WFO ให้สร้างว่างไว้
    df_bkf["wfo_id"] = np.nan
    bkf_wfo = "wfo_id"

# o7_spec  = find_col(df_o7.columns, ["specific_name"])
# o7_wfo   = find_col(df_o7.columns, ["wfo_id","wfo"])
# o7_full  = find_col(df_o7.columns, ["wfo_full_name"])

# c_spec   = find_col(df_c.columns, ["specific_name"])
# c_wfo    = find_col(df_c.columns, ["wfo_id"])
# c_full   = find_col(df_c.columns, ["wfo_full_name"])

In [5]:
# ========= NORMALIZE KEY & WFO =========
df_bkf["specific_name_clean"] = df_bkf[bkf_spec].apply(norm)
if bkf_url: df_bkf["species_url_clean"] = df_bkf[bkf_url].apply(norm)

# if o7_spec: df_o7["specific_name_clean"] = df_o7[o7_spec].apply(norm)
# if c_spec:  df_c["specific_name_clean"]  = df_c[c_spec].apply(norm)

# ทำความสะอาด WFO ในทุกตารางที่มี
df_bkf[bkf_wfo] = nonempty_series_like(df_bkf, bkf_wfo).apply(wfo_clean)
# if o7_wfo:
#     df_o7[o7_wfo] = nonempty_series_like(df_o7, o7_wfo).apply(wfo_clean)
# if c_wfo:
#     df_c[c_wfo]   = nonempty_series_like(df_c, c_wfo).apply(wfo_clean)

### STEP (0) : **Missing `wfo_id` Check**

In [6]:
# ========= STEP 0: BASELINE =========
missing_before = df_bkf[bkf_wfo].eq("") | df_bkf[bkf_wfo].isna()
print(f"[BASELINE] Missing WFO (BKF): {int(missing_before.sum())} / {len(df_bkf)}")

[BASELINE] Missing WFO (BKF): 898 / 7469


>   นับจำนวนแถวใน BKF ที่ยัง ไม่มี wfo_id ("" หรือ NaN).

In [7]:
# แถวที่ยังไม่มี wfo_id (missing)
df_bkf_missing = df_bkf[missing_before].copy()
print(f"Missing WFO (BKF): {len(df_bkf_missing)} แถว")

# แถวที่มี wfo_id แล้ว (not missing)
df_bkf_present = df_bkf[~missing_before].copy()
print(f"มี WFO แล้ว: {len(df_bkf_present)} แถว")

# ตัวอย่างตรวจสอบ
print("\nตัวอย่าง missing:")
display(df_bkf_missing.head(3))

print("\nตัวอย่างที่มี WFO แล้ว:")
display(df_bkf_present.head(3))

Missing WFO (BKF): 898 แถว
มี WFO แล้ว: 6571 แถว

ตัวอย่าง missing:


,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean
0,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum siamense (Craib) Tardieu,This is currently accepted.,"PENINSULAR: Nakhon Si Thammarat, Songkhla (type).","Known from 3 localities: the type-locality, an...","In small mats on damp sandy ground, edge of ma...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:33.566587+00:00,Myriophyllum siamense (Craib) Tardieu,,myriophyllum siamense (craib) tardieu,https://botany.dnp.go.th/eflora/floraspecies.h...
4,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora mucronata Poir.,This is currently accepted.,SOUTH-EASTERN: Chanthaburi; SOUTH-WESTERN: Pra...,"In the Old World tropics, occurring from the c...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:19.615290+00:00,Rhizophora mucronata Poir.,,rhizophora mucronata poir.,https://botany.dnp.go.th/eflora/floraspecies.h...
5,2,Rhizophoraceae,2 Bruguiera,Bruguiera,2.0,Bruguiera gymnorhiza (L.) Savigny,This is currently accepted.,"SOUTH-EASTERN: Chanthaburi, Trat; PENINSULAR: ...","In the Old World tropics, from S & E Africa th...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:33.101996+00:00,Bruguiera gymnorhiza (L.) Savigny,,bruguiera gymnorhiza (l.) savigny,https://botany.dnp.go.th/eflora/floraspecies.h...



ตัวอย่างที่มี WFO แล้ว:


,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean
1,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum tetrandrum Roxb.,This is currently accepted.,SOUTH-WESTERN: Prachuap Khiri Khan (Bang Sapha...,"E India (type), Indochina, Malay Peninsula.","In rather shallow, open water of ditches, cana...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:36.633398+00:00,Myriophyllum tetrandrum Roxb.,wfo-0001261381,myriophyllum tetrandrum roxb.,https://botany.dnp.go.th/eflora/floraspecies.h...
2,2,Haloragaceae,2 Haloragis,Haloragis,2.0,Haloragis micrantha (Thunb.) R.Br. ex Sieb. & ...,Gonocarpus micranthus,NORTH-EASTERN: Loei (Phu Kradueng).,"India, S & E China, N Vietnam, Hainan, Formosa...","In marshy mountain turf, moist places along mo...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:50.049475+00:00,Gonocarpus micranthus,wfo-0000715064,gonocarpus micranthus,https://botany.dnp.go.th/eflora/floraspecies.h...
3,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora apiculata Blume,This is currently accepted.,CENTRAL: Chon Buri (Si Racha); SOUTH-EASTERN: ...,In tropical SE Asia throughout Malesia (type) ...,Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:15.874773+00:00,Rhizophora apiculata Blume,wfo-0001131596,rhizophora apiculata blume,https://botany.dnp.go.th/eflora/floraspecies.h...


In [11]:
df_bkf_missing.to_csv("../Processed data/bkf_missing_wfo.csv", index=False)
df_bkf_present.to_csv("../Processed data/bkf_present_wfo.csv", index=False)
print("✅ Saved -> bkf_missing_wfo.csv และ bkf_present_wfo.csv")

✅ Saved -> bkf_missing_wfo.csv และ bkf_present_wfo.csv


In [12]:
df_bkf_present['wfo_id'].nunique()

6571

In [40]:
# -*- coding: utf-8 -*-
# ใช้ตัวแปรเดิมจากโน้ตบุ๊ก: df_bkf_present, bkf_wfo, bkf_full
import os, time, random
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, WebDriverException

# =========================================================
# CONFIG
# =========================================================
BASE_URL     = "https://www.worldfloraonline.org/taxon/"
MAX_ATTEMPTS = 4            # จำนวนครั้ง retry ต่อหนึ่งไอดี
WAIT_SEC     = 20           # เวลา wait DOM
SAVE_EVERY   = 20           # เซฟ progress/cache ทุก N ไอดี
SLEEP_MIN    = 4.0          # หน่วงสุ่มเพื่อสุภาพ (กันโดน rate limit)
SLEEP_MAX    = 9.0

# โฟลเดอร์สำหรับบันทึกผล (แก้ให้ตรงเครื่องคุณ)
PROCESSED_DIR = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
#os.makedirs(PROCESSED_DIR, exist_ok=True)
CACHE_CSV    = os.path.join(PROCESSED_DIR, "bkf_present_wf_cache.csv")
PROGRESS_CSV = os.path.join(PROCESSED_DIR, "bkf_present_wf_progress.csv")
OUT_CSV      = os.path.join(PROCESSED_DIR, "bkf_present_wf.csv")

# =========================================================
# SANITY CHECKS + เตรียมคอลัมน์เป้าหมาย
# =========================================================
# bkf_wfo ต้องเป็นชื่อคอลัมน์ (string)
if not isinstance(bkf_wfo, str):
    raise TypeError(f"bkf_wfo ควรเป็นชื่อคอลัมน์แบบ str แต่ได้ {type(bkf_wfo)}")

# ชื่อคอลัมน์สำหรับเก็บชื่อเต็ม
col_full = bkf_full if bkf_full else "wfo_full_name"
if col_full not in df_bkf_present.columns:
    df_bkf_present[col_full] = np.nan

def nonempty(s):
    return s.astype(str).str.strip().ne("")

# =========================================================
# เลือก "ทุก" WFO ID ที่ไม่ซ้ำใน df_bkf_present
# (ไม่กรองตาม miss_name เพื่อให้ดึงครบ 6571)
# =========================================================
ids_all_unique = (df_bkf_present[bkf_wfo]
                  .dropna().astype(str).str.strip()
                  .drop_duplicates()
                  .tolist())
print(f"🔢 Unique WFO IDs in df_bkf_present: {len(ids_all_unique)}")

# =========================================================
# โหลด/สร้าง CACHE: เก็บผล scrape (ok/err) เพื่อลดการยิงซ้ำ
# =========================================================
if os.path.exists(CACHE_CSV):
    cache_df = pd.read_csv(CACHE_CSV)
    if bkf_wfo not in cache_df.columns:
        # พยายาม rename คอลัมน์ที่ใกล้เคียง
        for alt in ["wfo_id","wfo","wfoid","wfo_code"]:
            if alt in cache_df.columns:
                cache_df = cache_df.rename(columns={alt: bkf_wfo})
                break
else:
    cache_df = pd.DataFrame(columns=[bkf_wfo, "scraped_name", "scrape_status"])

done_ok = set(cache_df.loc[cache_df["scrape_status"]=="ok", bkf_wfo]) if not cache_df.empty else set()
ids_to_scrape = [i for i in ids_all_unique if i not in done_ok]
print(f"🧭 To do after cache check: {len(ids_to_scrape)}")

# =========================================================
# ตั้งค่า Selenium (headless + polite UA)
# =========================================================
chrome_opts = Options()
chrome_opts.add_argument("--headless=new")      # comment บรรทัดนี้หากอยากเห็นหน้าจอจริง
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")
chrome_opts.add_argument("--disable-gpu")
chrome_opts.add_argument("--disable-blink-features=AutomationControlled")
chrome_opts.add_argument("--window-size=1280,900")
chrome_opts.add_argument("--lang=en-US")
chrome_opts.add_argument("--disable-features=NetworkService")
chrome_opts.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_opts)
driver.set_page_load_timeout(WAIT_SEC + 5)

# =========================================================
# Helpers
# =========================================================
def extract_name_from_html(html: str):
    soup = BeautifulSoup(html, "lxml")
    sci = soup.select_one("em[itemprop='scientificName'].taxonName")
    if not sci:
        sci = soup.select_one("em[itemprop='scientificName']") or soup.select_one("em.taxonName")
    auth = soup.select_one("span[itemprop='scientificNameAuthorship']")
    if not sci:
        return None
    sci_name  = sci.get_text(strip=True)
    auth_name = auth.get_text(strip=True) if auth else ""
    return f"{sci_name} {auth_name}".strip()

def polite_pause(extra=0.0):
    # หน่วงเวลาแบบสุ่มทุกครั้ง
    time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX) + extra)

def scrape_one_wfo(wfo_id: str):
    url = BASE_URL + str(wfo_id).strip()
    attempt = 0
    last_err = "unknown_error"
    while attempt < MAX_ATTEMPTS:
        attempt += 1
        try:
            driver.get(url)
            WebDriverWait(driver, WAIT_SEC).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "em[itemprop='scientificName'], em.taxonName"))
            )
            time.sleep(0.8)  # เผื่อหน้าโหลดช้า
            full_name = extract_name_from_html(driver.page_source)
            if full_name:
                return full_name, "ok"
            raise TimeoutException("no_scientificName_tag")
        except (TimeoutException, WebDriverException) as e:
            last_err = f"retry_{attempt}: {e}"
            # backoff เพิ่มเมื่อ error
            backoff = min(2.0 * attempt, 8.0) + random.uniform(0.7, 1.7)
            if any(k in str(e) for k in ["ERR_CONNEC", "ERR_TIMED_OUT", "ERR_FAILED"]):
                backoff += 2.0
            polite_pause(extra=backoff)
        except Exception as e:
            return None, f"error: {e}"
    return None, last_err

# =========================================================
# RUN SCRAPE
# =========================================================
progress_rows = []
try:
    total = len(ids_to_scrape)
    print(f"🚀 Start scraping {total} WFO pages (polite mode)")
    for i, wfo_id in enumerate(ids_to_scrape, start=1):
        wfo_id = str(wfo_id).strip()
        if not wfo_id or wfo_id.lower() == "nan":
            progress_rows.append({bkf_wfo: wfo_id, "scraped_name": "", "scrape_status": "empty_wfo_id"})
            continue

        full_name, status = scrape_one_wfo(wfo_id)
        progress_rows.append({bkf_wfo: wfo_id, "scraped_name": full_name or "", "scrape_status": status})

        # หน่วงแบบสุภาพทุกครั้ง
        polite_pause()

        # Save cache/progress เป็นช่วง ๆ
        if (i % SAVE_EVERY == 0) or (i == total):
            new_df = pd.DataFrame(progress_rows)
            cache_df = pd.concat([cache_df, new_df], ignore_index=True)
            # ให้สถานะ ok ชนะกรณีซ้ำ
            cache_df.sort_values(by=["scrape_status"], key=lambda s: s.ne("ok"), inplace=True)
            cache_df = cache_df.drop_duplicates(subset=[bkf_wfo], keep="first")
            cache_df.to_csv(CACHE_CSV, index=False)
            new_df.to_csv(PROGRESS_CSV, index=False)
            print(f"💾 Progress saved ({i}/{total}) -> {PROGRESS_CSV}")
            progress_rows = []
finally:
    driver.quit()

# =========================================================
# MERGE กลับเข้า df_bkf_present แล้วเติมชื่อให้ครบทุกแถว
# =========================================================
ok_map = cache_df.loc[cache_df["scrape_status"]=="ok", [bkf_wfo, "scraped_name"]].rename(
    columns={"scraped_name": "__wfo_full_name_scraped__"}
)

# merge แบบ left เพื่อได้คอลัมน์ชื่อ scraped ชั่วคราว
df_bkf_present = df_bkf_present.merge(ok_map, on=bkf_wfo, how="left")

# เติมชื่อเต็มเฉพาะที่ยังว่าง/ว่างเปล่า
fill_mask = df_bkf_present[col_full].isna() | (~nonempty(df_bkf_present[col_full]))
df_bkf_present.loc[fill_mask, col_full] = df_bkf_present.loc[fill_mask, "__wfo_full_name_scraped__"]

# ลบคอลัมน์ช่วย
df_bkf_present.drop(columns=["__wfo_full_name_scraped__"], inplace=True)

# =========================================================
# SAVE ออกไฟล์ในโฟลเดอร์ Processed data
# =========================================================
df_bkf_present.to_csv(OUT_CSV, index=False)
print(f"✅ DONE. Saved df_bkf_present (with '{col_full}') -> {OUT_CSV}")

# ออปชัน: รายงานจำนวนแถวที่เติมชื่อได้ใหม่จากรอบนี้
already_had = nonempty(df_bkf_present[col_full]).sum()
print(f"✨ Rows with non-empty '{col_full}': {already_had} / {len(df_bkf_present)}")

🔢 Unique WFO IDs in df_bkf_present: 6571
🧭 To do after cache check: 3561
🚀 Start scraping 3561 WFO pages (polite mode)


KeyboardInterrupt: 

In [52]:
# -*- coding: utf-8 -*-
# ============= WFO FULL NAME SCRAPER — FAST, POLITE, RESUMABLE =============
# ใช้ตัวแปรเดิมจากโน้ตบุ๊ก: df_bkf_present, bkf_wfo, bkf_full

import os, sys, time, random, uuid, shutil
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# ======================================================================
# CONFIG (ปรับให้เร็วขึ้น แต่ยังสุภาพพอไม่โดนบล็อก)
# ======================================================================
BASE_URL     = "https://www.worldfloraonline.org/taxon/"
MAX_ATTEMPTS = 4            # retry ต่อไอดี
WAIT_SEC     = 18           # รอ DOM
SAVE_EVERY   = 10           # << เซฟทุก 10 แถว
SLEEP_MIN    = 2.0          # << พักสุภาพสั้นลง
SLEEP_MAX    = 5.0

# โฟลเดอร์ผลลัพธ์ (ปรับ path ให้ตรงเครื่องคุณแล้วก่อนหน้า)
PROCESSED_DIR = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
CACHE_CSV     = os.path.join(PROCESSED_DIR, "bkf_present_wf_cache.csv")
PROGRESS_CSV  = os.path.join(PROCESSED_DIR, "bkf_present_wf_progress.csv")
OUT_CSV       = os.path.join(PROCESSED_DIR, "bkf_present_wf.csv")

# ======================================================================
# SANITY + เตรียมคอลัมน์เป้าหมาย
# ======================================================================
if not isinstance(bkf_wfo, str):
    raise TypeError(f"bkf_wfo ควรเป็นชื่อคอลัมน์แบบ str แต่ได้ {type(bkf_wfo)}")

col_full = bkf_full if bkf_full else "wfo_full_name"
if col_full not in df_bkf_present.columns:
    df_bkf_present[col_full] = np.nan

def nonempty(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().ne("")

# ======================================================================
# เตรียมรายการ WFO ที่จะดึง (เอา “ทั้งหมด” เพื่อให้เติมชื่อครบ 6571)
# ======================================================================
ids_all_unique = (df_bkf_present[bkf_wfo]
                  .dropna().astype(str).str.strip()
                  .drop_duplicates()
                  .tolist())
print(f"🔢 Unique WFO IDs in df_bkf_present: {len(ids_all_unique)}")

# ======================================================================
# โหลด/สร้าง CACHE และคำนวณคิวที่ยังต้องดึง
# ======================================================================
if os.path.exists(CACHE_CSV):
    cache_df = pd.read_csv(CACHE_CSV)
    if bkf_wfo not in cache_df.columns:
        for alt in ["wfo_id","wfo","wfoid","wfo_code"]:
            if alt in cache_df.columns:
                cache_df = cache_df.rename(columns={alt: bkf_wfo})
                break
else:
    cache_df = pd.DataFrame(columns=[bkf_wfo, "scraped_name", "scrape_status"])

done_ok = set(cache_df.loc[cache_df["scrape_status"]=="ok", bkf_wfo]) if not cache_df.empty else set()
ids_to_scrape = [i for i in ids_all_unique if i not in done_ok]
print(f"🧭 To do after cache check: {len(ids_to_scrape)}")

# ======================================================================
# ตั้งค่า Selenium (headless + polite UA + ปิดโหลดรูปให้ไวขึ้น)
# ======================================================================
chrome_opts = Options()
chrome_opts.add_argument("--headless=new")
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")
chrome_opts.add_argument("--disable-gpu")
chrome_opts.add_argument("--disable-blink-features=AutomationControlled")
chrome_opts.add_argument("--window-size=1280,900")
chrome_opts.add_argument("--lang=en-US")
chrome_opts.add_argument("--disable-features=NetworkService")
chrome_opts.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

# ปิดโหลดรูปเพื่อลดเวลาโหลดหน้า
chrome_opts.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2
})

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_opts)
driver.set_page_load_timeout(WAIT_SEC + 5)

# ======================================================================
# Helpers
# ======================================================================
def atomic_write(df: pd.DataFrame, path: str):
    """เขียนไฟล์แบบ atomic กันไฟล์ค้างครึ่ง ถ้า interrupt"""
    tmp = f"{path}.tmp_{uuid.uuid4().hex}"
    df.to_csv(tmp, index=False)
    shutil.move(tmp, path)

def extract_name_from_html(html: str):
    soup = BeautifulSoup(html, "lxml")
    sci = soup.select_one("em[itemprop='scientificName'].taxonName")
    if not sci:
        sci = soup.select_one("em[itemprop='scientificName']") or soup.select_one("em.taxonName")
    auth = soup.select_one("span[itemprop='scientificNameAuthorship']")
    if not sci:
        return None
    sci_name  = sci.get_text(strip=True)
    auth_name = auth.get_text(strip=True) if auth else ""
    return f"{sci_name} {auth_name}".strip()

def polite_pause(extra=0.0):
    time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX) + extra)

def scrape_one_wfo(wfo_id: str):
    url = BASE_URL + str(wfo_id).strip()
    attempt = 0
    last_err = "unknown_error"
    while attempt < MAX_ATTEMPTS:
        attempt += 1
        try:
            driver.get(url)
            WebDriverWait(driver, WAIT_SEC).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "em[itemprop='scientificName'], em.taxonName"))
            )
            time.sleep(0.4)  # เผื่อหน้าโหลดช้า (ลดลงเพื่อความเร็ว)
            full_name = extract_name_from_html(driver.page_source)
            if full_name:
                return full_name, "ok"
            raise TimeoutException("no_scientificName_tag")
        except (TimeoutException, WebDriverException) as e:
            last_err = f"retry_{attempt}: {e}"
            backoff = min(2.0 * attempt, 8.0) + random.uniform(0.5, 1.2)
            if any(k in str(e) for k in ["ERR_CONNEC", "ERR_TIMED_OUT", "ERR_FAILED",
                                          "ERR_CONNECTION_RESET","ERR_CONNECTION_CLOSED"]):
                backoff += 1.5
            polite_pause(extra=backoff)
        except Exception as e:
            return None, f"error: {e}"
    return None, last_err

# ======================================================================
# RUN SCRAPE
# ======================================================================
progress_rows = []
try:
    total = len(ids_to_scrape)
    print(f"🚀 Start scraping {total} WFO pages (polite mode)")
    for i, wfo_id in enumerate(ids_to_scrape, start=1):
        wfo_id = str(wfo_id).strip()
        if not wfo_id or wfo_id.lower() == "nan":
            progress_rows.append({bkf_wfo: wfo_id, "scraped_name": "", "scrape_status": "empty_wfo_id"})
            continue

        full_name, status = scrape_one_wfo(wfo_id)
        progress_rows.append({bkf_wfo: wfo_id, "scraped_name": full_name or "", "scrape_status": status})

        # หน่วงแบบสุภาพทุกครั้ง
        polite_pause()

        # Save cache/progress ทุก 10 แถว (หรือแถวสุดท้าย)
        if (i % SAVE_EVERY == 0) or (i == total):
            new_df = pd.DataFrame(progress_rows)

            # รวมเข้า cache และให้สถานะ ok ชนะกรณีซ้ำ
            cache_df = pd.concat([cache_df, new_df], ignore_index=True)
            cache_df.sort_values(by=["scrape_status"], key=lambda s: s.ne("ok"), inplace=True)
            cache_df = cache_df.drop_duplicates(subset=[bkf_wfo], keep="first")

            # เขียนไฟล์แบบ atomic
            atomic_write(cache_df, CACHE_CSV)
            atomic_write(new_df, PROGRESS_CSV)

            # flush stdout ให้เห็น log ทันที
            sys.stdout.flush()
            print(f"💾 Progress saved ({i}/{total}) -> {PROGRESS_CSV}")

            progress_rows = []
finally:
    try:
        driver.quit()
    except Exception:
        pass

# ======================================================================
# MERGE กลับเข้า df_bkf_present แล้วเติมชื่อให้ครบทุกแถว
# ======================================================================
ok_map = cache_df.loc[cache_df["scrape_status"]=="ok", [bkf_wfo, "scraped_name"]].rename(
    columns={"scraped_name": "__wfo_full_name_scraped__"}
)

# merge แบบ left เพื่อได้คอลัมน์ชื่อ scraped ชั่วคราว
df_bkf_present = df_bkf_present.merge(ok_map, on=bkf_wfo, how="left")

# เติมชื่อเต็มเฉพาะที่ยังว่าง/ว่างเปล่า
fill_mask = df_bkf_present[col_full].isna() | (~nonempty(df_bkf_present[col_full]))
df_bkf_present.loc[fill_mask, col_full] = df_bkf_present.loc[fill_mask, "__wfo_full_name_scraped__"]

# ลบคอลัมน์ช่วย
df_bkf_present.drop(columns=["__wfo_full_name_scraped__"], inplace=True, errors="ignore")

# ======================================================================
# SAVE ออกไฟล์ในโฟลเดอร์ Processed data
# ======================================================================
atomic_write(df_bkf_present, OUT_CSV)
print(f"✅ DONE. Saved df_bkf_present (with '{col_full}') -> {OUT_CSV}")

# รายงานจำนวนแถวที่มีชื่อเต็ม
rows_with_full = nonempty(df_bkf_present[col_full]).sum()
print(f"✨ Rows with non-empty '{col_full}': {rows_with_full} / {len(df_bkf_present)}")
# ======================================================================

🔢 Unique WFO IDs in df_bkf_present: 6571
🧭 To do after cache check: 7
🚀 Start scraping 7 WFO pages (polite mode)
💾 Progress saved (7/7) -> /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_present_wf_progress.csv
✅ DONE. Saved df_bkf_present (with 'accepted_name') -> /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_present_wf.csv
✨ Rows with non-empty 'accepted_name': 6571 / 6571


## Mapping Name From bkf_present_cache.csv to bkf_present_wfo.csv

In [58]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os

# ======================
# 📂 PATH CONFIG
# ======================
BASE_DIR    = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
INPUT_MAIN  = os.path.join(BASE_DIR, "bkf_present_wfo.csv")
INPUT_FIX   = os.path.join(BASE_DIR, "bkf_present_wf_cache_fix.csv")  # มี final_wfo_id และสถานะแก้ 7 แถว
OUTPUT_FILE = os.path.join(BASE_DIR, "bkf_present_wfo_with_fullname.csv")

# ======================
# 🔧 Helpers
# ======================
def norm_id(s: pd.Series) -> pd.Series:
    """ทำความสะอาดคีย์สำหรับ join ให้แมตช์ได้แม่นขึ้น"""
    return s.astype(str).str.strip().str.lower()

def nonempty(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().ne("") & s.notna()

# ======================
# 🧩 LOAD
# ======================
df_main = pd.read_csv(INPUT_MAIN)
df_fix  = pd.read_csv(INPUT_FIX)

required_fix_cols = {"wfo_id", "scrape_status", "scraped_name", "final_wfo_id"}
missing = required_fix_cols - set(df_fix.columns)
if missing:
    raise ValueError(f"ไฟล์ cache_fix ขาดคอลัมน์จำเป็น: {missing}")

if "wfo_id" not in df_main.columns:
    raise ValueError("ไฟล์หลักไม่มีคอลัมน์ 'wfo_id'")

# ======================
# 🧽 PREPARE FIX (กันซ้ำ เก็บค่าล่าสุด)
# ======================
# บางครั้งไฟล์ manual fix อาจมีหลายแถวต่อ wfo_id — เก็บแถวล่าสุดไว้
df_fix = (
    df_fix
    .copy()
    .assign(__key_wfo__=norm_id(df_fix["wfo_id"]))
    .drop_duplicates(subset=["__key_wfo__"], keep="last")
)

# สร้างคีย์ในไฟล์หลัก
df_main = df_main.copy()
df_main["__key_wfo__"] = norm_id(df_main["wfo_id"])

# ======================
# 🔗 MERGE เพื่อรู้สถานะและชื่อจาก cache_fix
# ======================
df_merged = df_main.merge(
    df_fix[["__key_wfo__", "scrape_status", "scraped_name", "final_wfo_id"]],
    on="__key_wfo__",
    how="left"
)

# ======================
# 🧠 เติม wfo_full_name
# เงื่อนไข:
# 1) ใช้ scraped_name ที่ได้มาไม่ว่าจาก ok หรือ not-ok (เพราะคุณขอ "เอาชื่อมา" สำหรับ not-ok ด้วย)
# 2) เฉพาะแถวที่ scraped_name ไม่ว่างเท่านั้นที่เติม
# ======================
if "wfo_full_name" not in df_merged.columns:
    df_merged["wfo_full_name"] = np.nan

has_name = nonempty(df_merged["scraped_name"])
df_merged.loc[has_name, "wfo_full_name"] = df_merged.loc[has_name, "scraped_name"]

# ======================
# 🔁 UPDATE wfo_id เป็น final_wfo_id สำหรับแถวที่ scrape_status != ok
# - เก็บ old_wfo_id ไว้ตรวจสอบ
# - เปลี่ยนเฉพาะแถวที่มี final_wfo_id ไม่ว่าง
# ======================
need_replace = (df_merged["scrape_status"].fillna("").ne("ok")) & nonempty(df_merged["final_wfo_id"])

# เก็บของเดิมก่อน (ครั้งแรกเท่านั้น)
if "old_wfo_id" not in df_merged.columns:
    df_merged["old_wfo_id"] = np.where(need_replace, df_merged["wfo_id"], np.nan)

# เปลี่ยน wfo_id เป็น final_wfo_id
df_merged.loc[need_replace, "wfo_id"] = df_merged.loc[need_replace, "final_wfo_id"]

# ======================
# 🧹 ทำความสะอาดคอลัมน์ช่วย
# ======================
df_merged.drop(columns=["__key_wfo__", "scraped_name", "scrape_status", "final_wfo_id"], inplace=True, errors="ignore")

# ======================
# 📊 รายงานสั้น ๆ
# ======================
n_full = int(nonempty(df_merged["wfo_full_name"]).sum())
n_replaced = int(need_replace.sum())
print(f"✨ Rows filled wfo_full_name: {n_full} / {len(df_merged)}")
print(f"🔁 Rows updated wfo_id → final_wfo_id (from not-ok): {n_replaced}")

# ======================
# 💾 บันทึกผล
# ======================
df_merged.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Done! Saved → {OUTPUT_FILE}")

✨ Rows filled wfo_full_name: 6571 / 6571
🔁 Rows updated wfo_id → final_wfo_id (from not-ok): 7
✅ Done! Saved → /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_present_wfo_with_fullname.csv


/var/folders/hb/q_224lns0mlcx0n865_2ylqc0000gn/T/ipykernel_97534/2979960506.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Myriophyllum tetrandrum Roxb.'
 'Haloragis micrantha R.Br. ex Siebold & Zucc.'
 'Rhizophora apiculata Blume' ... 'Popowia fusca King'
 'Popowia pisocarpa (Blume) Endl. ex Walp.'
 'Marsypopetalum modestum (Pierre) B.Xue & R.M.K.Saunders']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_merged.loc[has_name, "wfo_full_name"] = df_merged.loc[has_name, "scraped_name"]


In [61]:
#df_merged["wfo_full_name"].isna().sum()
df_merged

,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean,wfo_full_name,old_wfo_id
0,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum tetrandrum Roxb.,This is currently accepted.,SOUTH-WESTERN: Prachuap Khiri Khan (Bang Sapha...,"E India (type), Indochina, Malay Peninsula.","In rather shallow, open water of ditches, cana...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:36.633398+00:00,Myriophyllum tetrandrum Roxb.,wfo-0001261381,myriophyllum tetrandrum roxb.,https://botany.dnp.go.th/eflora/floraspecies.h...,Myriophyllum tetrandrum Roxb.,NaN
1,2,Haloragaceae,2 Haloragis,Haloragis,2.0,Haloragis micrantha (Thunb.) R.Br. ex Sieb. & ...,Gonocarpus micranthus,NORTH-EASTERN: Loei (Phu Kradueng).,"India, S & E China, N Vietnam, Hainan, Formosa...","In marshy mountain turf, moist places along mo...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:50.049475+00:00,Gonocarpus micranthus,wfo-0000715064,gonocarpus micranthus,https://botany.dnp.go.th/eflora/floraspecies.h...,Haloragis micrantha R.Br. ex Siebold & Zucc.,NaN
2,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora apiculata Blume,This is currently accepted.,CENTRAL: Chon Buri (Si Racha); SOUTH-EASTERN: ...,In tropical SE Asia throughout Malesia (type) ...,Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:15.874773+00:00,Rhizophora apiculata Blume,wfo-0001131596,rhizophora apiculata blume,https://botany.dnp.go.th/eflora/floraspecies.h...,Rhizophora apiculata Blume,NaN
3,2,Rhizophoraceae,2 Bruguiera,Bruguiera,2.0,Bruguiera sexangula (Lour.) Poir.,This is currently accepted.,"CENTRAL: Phra Nakhon, Thon Buri; SOUTH-EASTERN...","Ceylon, SE Asia (Cochinchina – type), througho...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:36.621984+00:00,Bruguiera sexangula (Lour.) Poir.,wfo-0000572763,bruguiera sexangula (lour.) poir.,https://botany.dnp.go.th/eflora/floraspecies.h...,Bruguiera sexangula (Lour.) Poir.,NaN
4,2,Rhizophoraceae,2 Bruguiera,Bruguiera,2.0,Bruguiera parviflora (Roxb.) Wight & Arn. ex G...,This is currently accepted.,"PENINSULAR: Phuket, Ranong, Surat Thani, Satun...","SE Asia (type), throughout Malesia to Melanesi...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:39.905942+00:00,Bruguiera parviflora (Roxb.) Wight & Arn. ex G...,wfo-0000572758,bruguiera parviflora (roxb.) wight & arn. ex g...,https://botany.dnp.go.th/eflora/floraspecies.h...,Bruguiera parviflora (Roxb.) Wight & Arn. ex G...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6566,16,Annonaceae,6. Cyathocalyx,Cyathocalyx,6.0,Cyathocalyx harmandii (Finet & Gagnep.) R.J.Wa...,This is currently accepted.,EASTERN: Chaiyaphum; SOUTH-EASTERN: Chanthaburi.,Laos (type).,"Evergreen forests, 10–400 m alt. Flowering and...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-21T00:07:19.480188+00:00,Cyathocalyx harmandii (Finet & Gagnep.) R.J.Wa...,wfo-0000749030,cyathocalyx harmandii (finet & gagnep.) r.j.wa...,https://botany.dnp.go.th/eflora/floraspecies.h...,Cyathocalyx harmandii (Finet & Gagnep.) R.J.Wa...,NaN
6567,16,Annonaceae,6. Cyathocalyx,Cyat

เติมชื่อ ของพวกที่มี wfo_id เดิมเรียบร้อย

In [62]:

df_merged[df_merged["old_wfo_id"].notna()]

,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean,wfo_full_name,old_wfo_id
37,2,Rosaceae,10 Rubus,Rubus,10.0,Rubus alpestris Blume,This is currently accepted.,NORTHERN: Chiang Mai (Fang).,"N Vietnam, Java (type).","In lower montane forests, 1,000–2,000 m alt.",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:24:58.561938+00:00,Rubus alpestris Blume,wfo-0001228711,rubus alpestris blume,https://botany.dnp.go.th/eflora/floraspecies.h...,Phyllanthus myrtifolius (Wight) Müll.Arg.,wfo-0000990767
1070,4,Leguminosae-Mimosoideae,9 Albizia,Albizia,9.0,Albizia attopeuensis (Pierre) I.C.Nielsen,This is currently accepted.,"NORTH-EASTERN: Loei; EASTERN: Chaiyaphum, Ubon...","Laos (type), S Vietnam.",Evergreen forests and savannas on rocky ground...,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T20:31:12.090337+00:00,Albizia attopeuensis (Pierre) I.C.Nielsen,wfo-0001291160,albizia attopeuensis (pierre) i.c.nielsen,https://botany.dnp.go.th/eflora/floraspecies.h...,Viburnum inopinatum Craib,wfo-0000192577
3134,8,Euphorbiaceae,69. Phyllanthus,Phyllanthus,69.0,Phyllanthus myrtifolius (Wight) Müll.Arg.,This is currently accepted.,Cultivated.,Sri Lanka (type).,Cultivated.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-20T00:31:27.879381+00:00,Phyllanthus myrtifolius (Wight) Müll.Arg.,wfo-0000524983,phyllanthus myrtifolius (wight) müll.arg.,https://botany.dnp.go.th/eflora/floraspecies.h...,Albizia attopeuensis (Pierre) I.C.Nielsen,wfo-0000271586
3162,8,Euphorbiaceae,69. Phyllanthus,Phyllanthus,69.0,Phyllanthus elegans Wall. ex Müll.Arg.,This is currently accepted.,"NORTHERN: Chiang Mai (Mae Taeng), Lampang (Nga...","Burma (type), Indochina, Malaysia (Kedah).","In evergreen forests, 10–600 m alt.",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-20T00:32:37.402973+00:00,Phyllanthus elegans Wall. ex Müll.Arg.,wfo-0001049235,phyllanthus elegans wall. ex müll.arg.,https://botany.dnp.go.th/eflora/floraspecies.h...,Phyllanthus elegans Wall. ex Müll.Arg.,wfo-0000270922
3191,8,Euphorbiaceae,16. Breynia,Breynia,16.0,Breynia fruticosa (L.) Müll.Arg.,This is currently accepted.,"NORTH-EASTERN: Sakon Nakhon (Phu Phan NP), Muk...","China (type), Indochina.","Dry dipterocarp forests, secondary forests, de...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-20T00:36:54.167649+00:00,Breynia fruticosa (L.) Müll.Arg.,wfo-0000990767,breynia fruticosa (l.) müll.arg.,https://botany.dnp.go.th/eflora/floraspecies.h...,Rubus alpestris Blume,wfo-0000412593
3224,8,Euphorbiaceae,7. Aporosa,Aporosa,7.0,Aporosa planchoniana Baill. ex Müll.Arg.,This is currently accepted.,"EASTERN: Surin, Ubon Ratchatani; SOUTH-WESTERN...","Burma (type), Indochina.",Secondary and disturbed evergreen or mixed dec...,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-20T00:39:56.947917+00:00,Aporosa planchoniana Baill. ex Müll.Arg.,wfo-0001049253,aporosa planchoniana baill. ex müll.arg.,https://botany.dnp.go.th/eflora/floraspecies.h...,Breynia fruticosa (L.) Müll.Arg.,wfo-0000249451
4773,13,Adoxaceae,2. Viburnum,Viburnum,2.0,Viburnum inopinatum Craib,This is currently accepted.,"NORTHERN: Chiang Mai (Doi Suthep – type), Chia...","Myanmar, China, Laos, Vietnam.","

## BKF missing wfo_id 

In [63]:
df_bkf_missing.shape

(898, 18)

มี 82 แถวที่ไม่มีรายชื่อ เพราะอาจจะสุดที่ genus ค่ะ จาก 898

มี 157 ที่เป็น candidates อีก 650 คือมีชื่อแน่ๆ งั้นเราควรแทนชื่อแทนๆลงไปเลย 91 no_candidates found

646 ชื่อที่เติมไปแล้วแน่ๆ อีก 6 ชื่อคือ unplaced เช็คอีกรอบ

In [73]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os

# ========= PATHS (ปรับตามโครงของคุณ) =========
BASE_PROC = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
BASE_RAW  = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Raw data"

FILE_BKF  = os.path.join(BASE_PROC, "bkf_missing_wfo.csv")    # ไฟล์หลัก 898 แถว
FILE_OUT  = os.path.join(BASE_RAW,  "output_missing.csv")     # ผลจาก WFO matcher
OUT_FILE  = os.path.join(BASE_PROC, "bkf_missing_wfo_step1.csv")

# ========= helpers =========
def norm(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower()

def nonempty(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().ne("") & s.notna()

# ========= load =========
df_bkf = pd.read_csv(FILE_BKF)
df_out = pd.read_csv(FILE_OUT)

# ตรวจคอลัมน์จำเป็น
req_bkf = {"specific_name_clean"}
req_out = {"wfo_check", "input_name_string"}
miss_bkf = req_bkf - set(df_bkf.columns)
miss_out = req_out - set(df_out.columns)
if miss_bkf:
    raise ValueError(f"bkf_missing_wfo.csv ขาดคอลัมน์: {miss_bkf}")
if miss_out:
    raise ValueError(f"output_missing.csv ขาดคอลัมน์: {miss_out}")

# สร้างคีย์สำหรับจับคู่แถว (ชื่อทำความสะอาดแล้ว)
df_bkf["__key__"] = norm(df_bkf["specific_name_clean"])
df_out["__key__"] = norm(df_out["input_name_string"])

# ระบุแถวใน output_missing ที่เป็น No candidates found + input_name_string ว่าง
is_no_cand      = df_out["wfo_check"].astype(str).str.fullmatch(r"\s*No candidates found\s*", case=False)
is_input_empty  = ~nonempty(df_out["input_name_string"])
target_keys     = set(df_out.loc[is_no_cand & is_input_empty, "__key__"])

# ให้แน่ใจว่ามีคอลัมน์ปลายทาง
if "wfo_full_name" not in df_bkf.columns:
    df_bkf["wfo_full_name"] = np.nan
if "wfo_id" not in df_bkf.columns:
    df_bkf["wfo_id"] = ""

# ทำมาร์กแถวในไฟล์หลักที่จะอัปเดต
mask_bkf = df_bkf["__key__"].isin(target_keys)

# เก็บสถิติก่อนหน้า
before_id_nonempty   = int(nonempty(df_bkf["wfo_id"]).sum())
before_name_nonempty = int(nonempty(df_bkf["wfo_full_name"]).sum())

# อัปเดตตามกติกา
df_bkf.loc[mask_bkf, "wfo_full_name"] = "No specie name found"
df_bkf.loc[mask_bkf, "wfo_id"]        = ""   # ปล่อยว่าง

# เก็บสถิติหลังอัปเดต
after_id_nonempty   = int(nonempty(df_bkf["wfo_id"]).sum())
after_name_nonempty = int(nonempty(df_bkf["wfo_full_name"]).sum())

# บันทึกไฟล์ใหม่ (ไม่ทับของเดิม)
df_bkf.drop(columns=["__key__"], inplace=True, errors="ignore")
df_bkf.to_csv(OUT_FILE, index=False)

# รายงาน
print(f"✅ Saved → {OUT_FILE}")
print(f"✨ Updated rows (No candidates found & empty input): {int(mask_bkf.sum())}")
print(f"🆔 wfo_id non-empty: {before_id_nonempty} → {after_id_nonempty}")
print(f"🏷️ wfo_full_name non-empty: {before_name_nonempty} → {after_name_nonempty}")

✅ Saved → /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_missing_wfo_step1.csv
✨ Updated rows (No candidates found & empty input): 82
🆔 wfo_id non-empty: 0 → 0
🏷️ wfo_full_name non-empty: 0 → 82


/var/folders/hb/q_224lns0mlcx0n865_2ylqc0000gn/T/ipykernel_97534/3314856369.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'No specie name found' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_bkf.loc[mask_bkf, "wfo_full_name"] = "No specie name found"
/var/folders/hb/q_224lns0mlcx0n865_2ylqc0000gn/T/ipykernel_97534/3314856369.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_bkf.loc[mask_bkf, "wfo_id"]        = ""   # ปล่อยว่าง


In [79]:
no_specie_rows = df_bkf[df_bkf["wfo_full_name"] == "No specie name found"]
display(no_specie_rows)

,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean,wfo_full_name
11,2,Rosaceae,10 Rubus,Rubus,10.0,NaN,NaN,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:25:59.824192+00:00,NaN,,NaN,https://botany.dnp.go.th/eflora/floraspecies.h...,No specie name found
505,2,Rosaceae,11 Fragaria,Fragaria,11.0,NaN,NaN,NaN,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,NaN,2025-10-20T16:53:04.162781+00:00,NaN,,NaN,NaN,No specie name found
506,2,Theaceae,2,NaN,2.0,NaN,NaN,NaN,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,NaN,2025-10-20T16:59:01.488529+00:00,NaN,,NaN,NaN,No specie name found
507,5,Opiliaceae,2 Melientha,Melientha,2.0,NaN,NaN,NaN,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,NaN,2025-10-20T17:39:32.811762+00:00,NaN,,NaN,NaN,No specie name found
508,5,Opiliaceae,4,NaN,4.0,NaN,NaN,NaN,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,NaN,2025-10-20T17:39:41.450956+00:00,NaN,,NaN,NaN,No specie name found
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,15,Rubiaceae,88. Porterandia,Porterandia,88.0,NaN,NaN,NaN,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,NaN,2025-10-20T23:00:16.723313+00:00,NaN,,NaN,NaN,No specie name found
882,15,Rubiaceae,99. Tarennoidea,Tarennoidea,99.0,NaN,NaN,NaN,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,NaN,2025-10-20T23:00:21.002202+00:00,NaN,,NaN,NaN,No specie name found
885,16,Zingiberaceae,5. Cautleya,Cautleya,5.0,NaN,NaN,NaN,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,NaN,2025-10-20T23:29:08.494849+00:00,NaN,,NaN,NaN,No specie name found
887,16,Zingiberaceae,3. Boesenbergia p.p.,Boesenbergia p.p.,3.0,NaN,NaN,NaN,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,NaN,2025-10-20T23:34:34.983910+00:00,NaN,,NaN,NaN,No specie name found


In [80]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os

# ========= PATHS =========
BASE_PROC = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
BASE_RAW  = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Raw data"

FILE_STEP1 = os.path.join(BASE_PROC, "bkf_missing_wfo_step1.csv")   # ผลจาก Step 1
FILE_FIX   = os.path.join(BASE_RAW,  "clean_no_candidate.csv")      # แมปชื่อที่เครื่องมือหาไม่เจอ
OUT_FILE   = os.path.join(BASE_PROC, "bkf_missing_wfo_step2.csv")   # ผลลัพธ์ Step 2

# ========= helpers =========
def norm(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower()

def nonempty(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().ne("") & s.notna()

# ========= load df_bkf (จากตัวแปรเดิม หรือไฟล์ step1) =========
try:
    df_bkf  # noqa: F821
except NameError:
    df_bkf = pd.read_csv(FILE_STEP1)

df_fix = pd.read_csv(FILE_FIX)

# ตรวจคอลัมน์จำเป็น
req_bkf = {"specific_name_clean", "wfo_id"}
req_fix = {"input_name_string", "wfo_id", "wfo_full_name"}
miss_bkf = req_bkf - set(df_bkf.columns)
miss_fix = req_fix - set(df_fix.columns)
if miss_bkf:
    raise ValueError(f"bkf_missing_wfo (step1) ขาดคอลัมน์: {miss_bkf}")
if miss_fix:
    raise ValueError(f"clean_no_candidate.csv ขาดคอลัมน์: {miss_fix}")

# สร้าง/รับประกันคอลัมน์ปลายทาง
if "wfo_full_name" not in df_bkf.columns:
    df_bkf["wfo_full_name"] = np.nan

# ========= เงื่อนไขเลือกแถวที่ต้องเติมใน df_bkf =========
# - ไม่มี wfo_id (ว่าง/NaN)
# - แต่ specific_name_clean ไม่ว่าง (มีชื่อให้ไปแมป)
need_fill = (~nonempty(df_bkf["wfo_id"])) & nonempty(df_bkf["specific_name_clean"])

# เตรียมคีย์แมป
df_bkf.loc[:, "__key__"] = norm(df_bkf["specific_name_clean"])
df_fix = df_fix.copy()
df_fix.loc[:, "__key__"] = norm(df_fix["input_name_string"])
# กันซ้ำในไฟล์แก้มือ: เก็บแถวล่าสุด
df_fix = df_fix.drop_duplicates(subset=["__key__"], keep="last")

# ========= Merge mapping จาก clean_no_candidate =========
to_fill = df_bkf.loc[need_fill, ["__key__", "wfo_id", "wfo_full_name"]].merge(
    df_fix[["__key__", "wfo_id", "wfo_full_name"]].rename(
        columns={"wfo_id": "wfo_id_fix", "wfo_full_name": "wfo_full_name_fix"}
    ),
    on="__key__", how="left"
)

# แถวที่มีของให้เติมจริง
can_fix = to_fill["wfo_id_fix"].notna() | to_fill["wfo_full_name_fix"].notna()

# ดึง index กลับไปอัปเดตใน df_bkf
idx_src = df_bkf.index[need_fill]
idx_to_update = idx_src[can_fix.values]  # จับคู่ตำแหน่งที่แมปได้

# สถิติก่อนเติม
before_id_nonempty   = int(nonempty(df_bkf["wfo_id"]).sum())
before_name_nonempty = int(nonempty(df_bkf["wfo_full_name"]).sum())

# เติมค่า
df_bkf.loc[idx_to_update, "wfo_id"]        = to_fill.loc[can_fix, "wfo_id_fix"].astype(str).values
df_bkf.loc[idx_to_update, "wfo_full_name"] = to_fill.loc[can_fix, "wfo_full_name_fix"].astype(str).values

# สถิติหลังเติม
after_id_nonempty   = int(nonempty(df_bkf["wfo_id"]).sum())
after_name_nonempty = int(nonempty(df_bkf["wfo_full_name"]).sum())

# ทำความสะอาดคอลัมน์ช่วย & บันทึก
df_bkf.drop(columns=["__key__"], inplace=True, errors="ignore")
df_bkf.to_csv(OUT_FILE, index=False)

# รายงาน
print(f"✅ Saved → {OUT_FILE}")
print(f"✨ Rows eligible for fill (no wfo_id but has specific_name_clean): {int(need_fill.sum())}")
print(f"🔁 Rows updated from clean_no_candidate: {int(can_fix.sum())}")
print(f"🆔 wfo_id non-empty: {before_id_nonempty} → {after_id_nonempty}")
print(f"🏷️ wfo_full_name non-empty: {before_name_nonempty} → {after_name_nonempty}")

✅ Saved → /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_missing_wfo_step2.csv
✨ Rows eligible for fill (no wfo_id but has specific_name_clean): 807
🔁 Rows updated from clean_no_candidate: 0
🆔 wfo_id non-empty: 9 → 9
🏷️ wfo_full_name non-empty: 91 → 91


In [81]:
df_bkf

,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean,wfo_full_name
0,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum siamense (Craib) Tardieu,This is currently accepted.,"PENINSULAR: Nakhon Si Thammarat, Songkhla (type).","Known from 3 localities: the type-locality, an...","In small mats on damp sandy ground, edge of ma...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:33.566587+00:00,Myriophyllum siamense (Craib) Tardieu,NaN,myriophyllum siamense (craib) tardieu,https://botany.dnp.go.th/eflora/floraspecies.h...,NaN
1,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora mucronata Poir.,This is currently accepted.,SOUTH-EASTERN: Chanthaburi; SOUTH-WESTERN: Pra...,"In the Old World tropics, occurring from the c...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:19.615290+00:00,Rhizophora mucronata Poir.,NaN,rhizophora mucronata poir.,https://botany.dnp.go.th/eflora/floraspecies.h...,NaN
2,2,Rhizophoraceae,2 Bruguiera,Bruguiera,2.0,Bruguiera gymnorhiza (L.) Savigny,This is currently accepted.,"SOUTH-EASTERN: Chanthaburi, Trat; PENINSULAR: ...","In the Old World tropics, from S & E Africa th...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:33.101996+00:00,Bruguiera gymnorhiza (L.) Savigny,NaN,bruguiera gymnorhiza (l.) savigny,https://botany.dnp.go.th/eflora/floraspecies.h...,NaN
3,2,Rhizophoraceae,3 Ceriops,Ceriops,3.0,Ceriops decandra (Griff.) Ding Hou,This is currently accepted.,"SOUTH-EASTERN: Chon Buri (Si Racha), Chathabur...","India (type), Burma, Cambodia, S Vietnam, Male...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:19:14.152028+00:00,Ceriops decandra (Griff.) Ding Hou,NaN,ceriops decandra (griff.) ding hou,https://botany.dnp.go.th/eflora/floraspecies.h...,NaN
4,2,Rosaceae,5 Stranvaesia,Stranvaesia,5.0,Stranvaesia nussia (Buch.-Ham. ex D.Don) Decne.,Photinia nussia,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-19T18:23:26.198148+00:00,Photinia nussia,NaN,photinia nussia,https://botany.dnp.go.th/eflora/floraspecies.h...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,16,Annonaceae,36. Uvaria,Uvaria,36.0,Uvaria ferruginea Buch.-Ham. ex Hook.f. & Thomson,This is currently accepted.,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-20T23:40:07.128790+00:00,Uvaria ferruginea Buch.-Ham. ex Hook.f. & Thomson,NaN,uvaria ferruginea buch.-ham. ex hook.f. & thomson,https://botany.dnp.go.th/eflora/floraspecies.h...,NaN
894,16,Annonaceae,36. Uvaria,Uvaria,36.0,Uvaria pauciovulata Hook.f. & Thomson,NaN,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-20T23:40:29.298457+00:00,Uvaria pauciovulata Hook.f. & Thomson,NaN,uvaria pauciovulata hook.f. & thomson,https://botany.dnp.go.th/eflora/floraspecies.h...,NaN
895,16,Annonaceae,36. Uvaria,Uvaria,36.0,"Uvaria wrayi (King) L.L.Zhou, Y.C.F.Su & R.M.K...",This is currently accepted.,e-Flora of Thailand,NaN,NaN,https://botany.dnp.g

no candidate จัดการแล้ว ต่อไปจัดการที่มันไม่มีแคนดิเดต

In [83]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import os

# ========= PATHS =========
BASE_PROC = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
BASE_RAW  = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Raw data"

FILE_DF_BKF = os.path.join(BASE_PROC, "bkf_missing_wfo_step2.csv")   # ผลล่าสุดจาก Step 2 (หรือใช้ df_bkf ที่มีอยู่แล้ว)
FILE_OUT    = os.path.join(BASE_RAW,  "output_missing.csv")
OUT_FILE    = os.path.join(BASE_PROC, "bkf_missing_wfo_step3.csv")

# ========= helpers =========
def norm(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower()

def nonempty(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().ne("") & s.notna()

# ========= load =========
try:
    df_bkf  # noqa
except NameError:
    df_bkf = pd.read_csv(FILE_DF_BKF)

df_out = pd.read_csv(FILE_OUT)

# ตรวจคอลัมน์จำเป็น
req_bkf = {"specific_name_clean", "wfo_id"}  # wfo_full_name อาจยังไม่มีก็จะสร้างให้
req_out = {"input_name_string", "wfo_check", "wfo_id", "wfo_full_name"}
miss_bkf = req_bkf - set(df_bkf.columns)
miss_out = req_out - set(df_out.columns)
if miss_bkf:
    raise ValueError(f"df_bkf ขาดคอลัมน์: {miss_bkf}")
if miss_out:
    raise ValueError(f"output_missing.csv ขาดคอลัมน์: {miss_out}")

# ให้แน่ใจว่ามีคอลัมน์เป้าหมาย
if "wfo_full_name" not in df_bkf.columns:
    df_bkf["wfo_full_name"] = np.nan

# ========= เตรียมคีย์สำหรับแมป =========
df_bkf["__key__"] = norm(df_bkf["specific_name_clean"])
df_out["__key__"] = norm(df_out["input_name_string"])

# ========= เลือกแถวใน output ที่ "ชัดเจน" =========
# เงื่อนไข: input_name_string ไม่ว่าง และ wfo_check ไม่ใช่ 'No candidates found' และไม่พูดถึง 'candidates'
wfo_check = df_out["wfo_check"].astype(str)
valid_out = (
    nonempty(df_out["input_name_string"]) &
    (~wfo_check.str.contains("candidates", case=False, na=False)) &
    (~wfo_check.str.fullmatch(r"\s*No candidates found\s*", case=False))
)

df_out_valid = (
    df_out.loc[valid_out, ["__key__", "wfo_id", "wfo_full_name"]]
          .dropna(subset=["__key__"])
          .drop_duplicates(subset=["__key__"], keep="last")
          .rename(columns={"wfo_id": "wfo_id_from_out", "wfo_full_name": "wfo_full_from_out"})
)

# ========= สร้าง mask ใน df_bkf สำหรับแถวที่ควรเติม (ยังว่าง) =========
need_fill = (~nonempty(df_bkf["wfo_id"])) | (~nonempty(df_bkf["wfo_full_name"]))

# ========= merge map แล้วอัปเดตเฉพาะแถวที่ยังว่าง =========
df_merge = df_bkf.merge(df_out_valid, on="__key__", how="left")

has_map_id   = nonempty(df_merge["wfo_id_from_out"])
has_map_name = nonempty(df_merge["wfo_full_from_out"])
can_update   = need_fill & (has_map_id | has_map_name)

# เก็บสถิติก่อน
before_id_nonempty   = int(nonempty(df_merge["wfo_id"]).sum())
before_name_nonempty = int(nonempty(df_merge["wfo_full_name"]).sum())

# เติมแบบไม่ทับค่าที่มีอยู่แล้ว (fill only where empty)
fill_id_mask   = can_update & (~nonempty(df_merge["wfo_id"]))   & has_map_id
fill_name_mask = can_update & (~nonempty(df_merge["wfo_full_name"])) & has_map_name

df_merge.loc[fill_id_mask,   "wfo_id"]        = df_merge.loc[fill_id_mask,   "wfo_id_from_out"].astype(str)
df_merge.loc[fill_name_mask, "wfo_full_name"] = df_merge.loc[fill_name_mask, "wfo_full_from_out"].astype(str)

# หลังเติม
after_id_nonempty   = int(nonempty(df_merge["wfo_id"]).sum())
after_name_nonempty = int(nonempty(df_merge["wfo_full_name"]).sum())
updated_rows = int((fill_id_mask | fill_name_mask).sum())

# ทำความสะอาดแล้วบันทึก
df_result = df_merge.drop(columns=["__key__", "wfo_id_from_out", "wfo_full_from_out"], errors="ignore")
df_result.to_csv(OUT_FILE, index=False)

print(f"✅ Saved → {OUT_FILE}")
print(f"✨ Eligible to fill: {int(need_fill.sum())} rows")
print(f"🔁 Updated rows from output_missing (definitive matches): {updated_rows}")
print(f"🆔 wfo_id non-empty: {before_id_nonempty} → {after_id_nonempty}")
print(f"🏷️ wfo_full_name non-empty: {before_name_nonempty} → {after_name_nonempty}")

✅ Saved → /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_missing_wfo_step3.csv
✨ Eligible to fill: 889 rows
🔁 Updated rows from output_missing (definitive matches): 650
🆔 wfo_id non-empty: 9 → 659
🏷️ wfo_full_name non-empty: 91 → 741


659 wfo_id = 650+9 ที่เติมชื่อแล้ว

In [84]:
df_result

,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean,wfo_full_name
0,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum siamense (Craib) Tardieu,This is currently accepted.,"PENINSULAR: Nakhon Si Thammarat, Songkhla (type).","Known from 3 localities: the type-locality, an...","In small mats on damp sandy ground, edge of ma...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:33.566587+00:00,Myriophyllum siamense (Craib) Tardieu,wfo-0000373567,myriophyllum siamense (craib) tardieu,https://botany.dnp.go.th/eflora/floraspecies.h...,Myriophyllum siamense (Craib) Tardieu
1,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora mucronata Poir.,This is currently accepted.,SOUTH-EASTERN: Chanthaburi; SOUTH-WESTERN: Pra...,"In the Old World tropics, occurring from the c...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:19.615290+00:00,Rhizophora mucronata Poir.,wfo-0001131556,rhizophora mucronata poir.,https://botany.dnp.go.th/eflora/floraspecies.h...,Rhizophora mucronata Poir.
2,2,Rhizophoraceae,2 Bruguiera,Bruguiera,2.0,Bruguiera gymnorhiza (L.) Savigny,This is currently accepted.,"SOUTH-EASTERN: Chanthaburi, Trat; PENINSULAR: ...","In the Old World tropics, from S & E Africa th...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:33.101996+00:00,Bruguiera gymnorhiza (L.) Savigny,NaN,bruguiera gymnorhiza (l.) savigny,https://botany.dnp.go.th/eflora/floraspecies.h...,NaN
3,2,Rhizophoraceae,3 Ceriops,Ceriops,3.0,Ceriops decandra (Griff.) Ding Hou,This is currently accepted.,"SOUTH-EASTERN: Chon Buri (Si Racha), Chathabur...","India (type), Burma, Cambodia, S Vietnam, Male...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:19:14.152028+00:00,Ceriops decandra (Griff.) Ding Hou,NaN,ceriops decandra (griff.) ding hou,https://botany.dnp.go.th/eflora/floraspecies.h...,NaN
4,2,Rosaceae,5 Stranvaesia,Stranvaesia,5.0,Stranvaesia nussia (Buch.-Ham. ex D.Don) Decne.,Photinia nussia,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-19T18:23:26.198148+00:00,Photinia nussia,wfo-0000995607,photinia nussia,https://botany.dnp.go.th/eflora/floraspecies.h...,Photinia nussia (Buch.-Ham. ex D.Don) Kalkman
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,16,Annonaceae,36. Uvaria,Uvaria,36.0,Uvaria ferruginea Buch.-Ham. ex Hook.f. & Thomson,This is currently accepted.,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-20T23:40:07.128790+00:00,Uvaria ferruginea Buch.-Ham. ex Hook.f. & Thomson,wfo-1000041775,uvaria ferruginea buch.-ham. ex hook.f. & thomson,https://botany.dnp.go.th/eflora/floraspecies.h...,Uvaria ferruginea Buch.-Ham. ex Hook.f. & Thomson
894,16,Annonaceae,36. Uvaria,Uvaria,36.0,Uvaria pauciovulata Hook.f. & Thomson,NaN,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-20T23:40:29.298457+00:00,Uvaria pauciovulata Hook.f. & Thomson,wfo-0001066292,uvaria pauciovulata hook.f. & thomson,https://botany.dnp.go.t

อีก 157 rows ที่ต้องจัดการเพราะเป็น candidates

In [85]:
# -*- coding: utf-8 -*-
"""
STEP 4 (no-overwrite):
อ่าน bkf_missing_wfo_step3.csv แล้วแมปกับ candidate_clean_ready.csv โดย:
1) ถ้า candidate.POWO_name ไม่ว่าง -> เติมลง df_main เฉพาะแถวที่ POWO_name ยังว่าง
2) ถ้า use_input_name_with_noid == TRUE -> ตั้ง wfo_full_name = input_name_string เฉพาะแถวที่ wfo_full_name ยังว่าง
   และ "ไม่" เคลียร์/ทับ wfo_id ถ้าเดิมมีค่าแล้ว (จะตั้งเป็นว่างเฉพาะกรณีที่มันว่างอยู่แล้วเท่านั้น)
3) ถ้า use_input_name_with_noid == FALSE -> เติม wfo_id / wfo_full_name จาก candidate
   เฉพาะช่องที่ยังว่างใน df_main เท่านั้น
"""

import pandas as pd
import numpy as np
import os

# =================== PATHS ===================
BASE_PROC = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data"
BASE_RAW  = "/Users/thanwaratkeratipasuwat/Desktop/dsi314/Raw data"

FILE_MAIN = os.path.join(BASE_PROC, "bkf_missing_wfo_step3.csv")
FILE_CAND = os.path.join(BASE_RAW,  "candidate_clean_ready.csv")
OUT_FILE  = os.path.join(BASE_PROC, "bkf_missing_wfo_step4.csv")

# =================== Helpers ===================
def norm(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().str.lower()

def nonempty(s: pd.Series) -> pd.Series:
    return s.astype(str).str.strip().ne("") & s.notna()

# =================== LOAD ===================
df_main = pd.read_csv(FILE_MAIN)
df_cand = pd.read_csv(FILE_CAND)

# ตรวจคอลัมน์จำเป็น
req_main = {"specific_name_clean"}
req_cand = {"input_name_string", "wfo_id", "wfo_full_name", "use_input_name_with_noid", "POWO_name"}
miss_main = req_main - set(df_main.columns)
miss_cand = req_cand - set(df_cand.columns)
if miss_main:
    raise ValueError(f"bkf_missing_wfo_step3.csv ขาดคอลัมน์: {miss_main}")
if miss_cand:
    raise ValueError(f"candidate_clean_ready.csv ขาดคอลัมน์: {miss_cand}")

# ให้แน่ใจว่ามีคอลัมน์ปลายทางใน df_main
if "wfo_id" not in df_main.columns:
    df_main["wfo_id"] = np.nan
if "wfo_full_name" not in df_main.columns:
    df_main["wfo_full_name"] = np.nan
if "POWO_name" not in df_main.columns:
    df_main["POWO_name"] = np.nan

# =================== BUILD KEYS & DEDUPE ===================
df_main = df_main.copy()
df_cand = df_cand.copy()

df_main["__key__"] = norm(df_main["specific_name_clean"])
df_cand["__key__"] = norm(df_cand["input_name_string"])

# กันซ้ำฝั่ง candidate: เก็บแถวสุดท้าย
df_cand = df_cand.drop_duplicates(subset=["__key__"], keep="last")

# =================== MERGE ===================
cols_map = ["__key__", "wfo_id", "wfo_full_name", "use_input_name_with_noid", "POWO_name", "input_name_string"]
dfm = df_main.merge(df_cand[cols_map], on="__key__", how="left", suffixes=("", "_cand"))

# แปลง use_input_name_with_noid -> bool
def to_bool(x):
    if pd.isna(x): return False
    if isinstance(x, (int, float)): return bool(int(x))
    s = str(x).strip().lower()
    return s in ("true", "1", "yes", "y")

dfm["__use_input_noid__"] = dfm["use_input_name_with_noid"].map(to_bool)

# =================== NO-OVERWRITE MASKS (ช่องว่างเท่านั้น) ===================
empty_id    = ~nonempty(dfm["wfo_id"])
empty_name  = ~nonempty(dfm["wfo_full_name"])
empty_powo  = ~nonempty(dfm["POWO_name"])

has_powo_c  = nonempty(dfm["POWO_name_cand"])
has_id_c    = nonempty(dfm["wfo_id_cand"])
has_name_c  = nonempty(dfm["wfo_full_name_cand"])
has_input_c = nonempty(dfm["input_name_string"])

# =================== RULE 1: POWO_name (เติมเมื่อว่างเท่านั้น) ===================
fill_powo_mask = empty_powo & has_powo_c
dfm.loc[fill_powo_mask, "POWO_name"] = dfm.loc[fill_powo_mask, "POWO_name_cand"]

# =================== RULE 2: use_input_name_with_noid == TRUE ===================
# เติมชื่อจาก input_name_string เฉพาะเมื่อชื่อยังว่าง
use_input_true = dfm["__use_input_noid__"] & has_input_c
fill_name_true = use_input_true & empty_name
dfm.loc[fill_name_true, "wfo_full_name"] = dfm.loc[fill_name_true, "input_name_string"].astype(str)

# ไม่ทับ wfo_id เดิม: ถ้าเดิมว่างอยู่ คงว่างไว้ (ไม่ต้องไป set ซ้ำ)
# (ถ้าอยาก "บังคับให้ว่าง" เมื่อ TRUE แต่ไม่ทับของเดิม ให้ใช้: dfm.loc[use_input_true & empty_id, "wfo_id"] = "")

# =================== RULE 3: use_input_name_with_noid == FALSE ===================
use_input_false = ~dfm["__use_input_noid__"]

# เติม id เฉพาะเมื่อ id ยังว่าง + candidate มี id
fill_id_false = use_input_false & empty_id & has_id_c
dfm.loc[fill_id_false, "wfo_id"] = dfm.loc[fill_id_false, "wfo_id_cand"].astype(str)

# เติมชื่อเฉพาะเมื่อชื่อยังว่าง + candidate มีชื่อ
fill_name_false = use_input_false & empty_name & has_name_c
dfm.loc[fill_name_false, "wfo_full_name"] = dfm.loc[fill_name_false, "wfo_full_name_cand"].astype(str)

# =================== SAVE ===================
df_result = dfm.drop(columns=[
    "__key__", "wfo_id_cand", "wfo_full_name_cand",
    "use_input_name_with_noid", "POWO_name_cand",
    "input_name_string", "__use_input_noid__"
], errors="ignore")

df_result.to_csv(OUT_FILE, index=False)

print(f"✅ Saved → {OUT_FILE}")
print(f"POWO_name filled (no-overwrite): {int(fill_powo_mask.sum())} rows")
print(f"use_input_name_with_noid == TRUE → names filled (no-overwrite): {int(fill_name_true.sum())} rows")
print(f"use_input_name_with_noid == FALSE → id filled: {int(fill_id_false.sum())}, name filled: {int(fill_name_false.sum())}")

✅ Saved → /Users/thanwaratkeratipasuwat/Desktop/dsi314/Processed data/bkf_missing_wfo_step4.csv
POWO_name filled (no-overwrite): 2 rows
use_input_name_with_noid == TRUE → names filled (no-overwrite): 25 rows
use_input_name_with_noid == FALSE → id filled: 131, name filled: 131


/var/folders/hb/q_224lns0mlcx0n865_2ylqc0000gn/T/ipykernel_97534/2024479339.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Stephania rotunda Lour.' 'Clematis siamensis J.R.Drumm. & Craib']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfm.loc[fill_powo_mask, "POWO_name"] = dfm.loc[fill_powo_mask, "POWO_name_cand"]


In [86]:
df_result

,volume,family_name,genus_name,genus_label,genus_index_parsed,species_scientific_name,accepted_name,thailand,distribution,ecology,family_url,genus_url,species_url,scraped_at,specific_name,wfo_id,specific_name_clean,species_url_clean,wfo_full_name,POWO_name
0,2,Haloragaceae,1 Myriophyllum,Myriophyllum,1.0,Myriophyllum siamense (Craib) Tardieu,This is currently accepted.,"PENINSULAR: Nakhon Si Thammarat, Songkhla (type).","Known from 3 localities: the type-locality, an...","In small mats on damp sandy ground, edge of ma...",https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:17:33.566587+00:00,Myriophyllum siamense (Craib) Tardieu,wfo-0000373567,myriophyllum siamense (craib) tardieu,https://botany.dnp.go.th/eflora/floraspecies.h...,Myriophyllum siamense (Craib) Tardieu,NaN
1,2,Rhizophoraceae,1 Rhizophora,Rhizophora,1.0,Rhizophora mucronata Poir.,This is currently accepted.,SOUTH-EASTERN: Chanthaburi; SOUTH-WESTERN: Pra...,"In the Old World tropics, occurring from the c...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:19.615290+00:00,Rhizophora mucronata Poir.,wfo-0001131556,rhizophora mucronata poir.,https://botany.dnp.go.th/eflora/floraspecies.h...,Rhizophora mucronata Poir.,NaN
2,2,Rhizophoraceae,2 Bruguiera,Bruguiera,2.0,Bruguiera gymnorhiza (L.) Savigny,This is currently accepted.,"SOUTH-EASTERN: Chanthaburi, Trat; PENINSULAR: ...","In the Old World tropics, from S & E Africa th...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:18:33.101996+00:00,Bruguiera gymnorhiza (L.) Savigny,wfo-0000572747,bruguiera gymnorhiza (l.) savigny,https://botany.dnp.go.th/eflora/floraspecies.h...,Bruguiera gymnorhiza (L.) Lam. ex Savigny,NaN
3,2,Rhizophoraceae,3 Ceriops,Ceriops,3.0,Ceriops decandra (Griff.) Ding Hou,This is currently accepted.,"SOUTH-EASTERN: Chon Buri (Si Racha), Chathabur...","India (type), Burma, Cambodia, S Vietnam, Male...",Mangrove forests.,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraspecies.h...,2025-10-19T18:19:14.152028+00:00,Ceriops decandra (Griff.) Ding Hou,wfo-0000597936,ceriops decandra (griff.) ding hou,https://botany.dnp.go.th/eflora/floraspecies.h...,Ceriops decandra (Griff.) W.Theob.,NaN
4,2,Rosaceae,5 Stranvaesia,Stranvaesia,5.0,Stranvaesia nussia (Buch.-Ham. ex D.Don) Decne.,Photinia nussia,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-19T18:23:26.198148+00:00,Photinia nussia,wfo-0000995607,photinia nussia,https://botany.dnp.go.th/eflora/floraspecies.h...,Photinia nussia (Buch.-Ham. ex D.Don) Kalkman,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,16,Annonaceae,36. Uvaria,Uvaria,36.0,Uvaria ferruginea Buch.-Ham. ex Hook.f. & Thomson,This is currently accepted.,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-20T23:40:07.128790+00:00,Uvaria ferruginea Buch.-Ham. ex Hook.f. & Thomson,wfo-1000041775,uvaria ferruginea buch.-ham. ex hook.f. & thomson,https://botany.dnp.go.th/eflora/floraspecies.h...,Uvaria ferruginea Buch.-Ham. ex Hook.f. & Thomson,NaN
894,16,Annonaceae,36. Uvaria,Uvaria,36.0,Uvaria pauciovulata Hook.f. & Thomson,NaN,e-Flora of Thailand,NaN,NaN,https://botany.dnp.go.th/eflora/florafamily.ht...,https://botany.dnp.go.th/eflora/floragenus.htm...,https://botany.dnp.go.th/eflora/floraSpecies.h...,2025-10-20T23:40:2